# Field Survey Campaign Progress

In [ ]:
# Parameters cell. Set defaults here
day_obs_min = 20250421
day_obs_max = 20250601

In [ ]:
import numpy as np
import pandas as pd
from astropy.time import Time
import matplotlib.pyplot as plt

In [ ]:
import os
os.environ["no_proxy"] += ",.consdb"

from lsst.summit.utils import ConsDbClient

client = ConsDbClient("http://consdb-pq.consdb:8080/consdb")

In [ ]:
instrument = 'lsstcam'

In [ ]:
bands = {'u': '#0c71ff', 'g': '#49be61', 'r': '#c61c00', 'i': '#ffc200', 'z': '#f341a2', 'y': '#5d0000'}

## All Visits

In [ ]:
visits_query = f'''
    SELECT 
        * 
    FROM 
        cdb_{instrument}.visit1 
    WHERE 
        day_obs >= {day_obs_min} AND day_obs <= {day_obs_max}
'''

visits = client.query(visits_query).to_pandas()

In [ ]:
visits['band'].value_counts().reindex(bands).fillna(0).astype(int)

In [ ]:
visits['img_type'].value_counts()

## Field Survey Science Program Visits

In [ ]:
visits_query = f'''
    SELECT 
        * 
    FROM 
        cdb_{instrument}.visit1 
    WHERE 
        science_program in ('BLOCK-365')
        AND day_obs >= {day_obs_min} AND day_obs <= {day_obs_max}
'''

visits = client.query(visits_query).to_pandas()

In [ ]:
print('Total number of science program visits = %i'%(len(visits)))

### Distribution of visit day_obs

In [ ]:
visits['day_obs'].value_counts().sort_index()

In [ ]:
def dayObsToMJD(day_obs):
    year = np.floor(day_obs / 10000).astype(int)
    month = np.floor((day_obs % 10000) / 100).astype(int)
    day = day_obs % 100
    time = Time({'year': year, 'month': month, 'day': day}, format='ymdhms')
    return time.mjd

In [ ]:
def MJDToDayObs(mjd):
    time = Time(mjd, format='mjd')
    return np.array([_.iso.split()[0].replace('-', '') for _ in time])

In [ ]:
mjd = dayObsToMJD(visits['day_obs'])
xticks = np.arange(np.min(mjd), np.max(mjd) + 2)[::2]
xtick_labels = MJDToDayObs(xticks)

plt.figure()
plt.scatter(dayObsToMJD(visits.value_counts('day_obs').index.values), 
            visits.value_counts('day_obs').values, c='black', label='Total')
for band in bands:
    selection = (visits['band'] == band)
    plt.scatter(dayObsToMJD(visits[selection].value_counts('day_obs').index.values), 
                visits[selection].value_counts('day_obs').values, 
                c=bands[band], label=band)

plt.legend(loc='upper left')
plt.xticks(xticks, xtick_labels, rotation=45.)
plt.ylabel('Visit Count')
plt.grid()
plt.tight_layout()

### Distribution of visits by target and band

In [ ]:
targets = np.unique(visits['target_name'])

In [ ]:
data_target_counts = {
    'target': [],
    'u': [],
    'g': [],
    'r': [],
    'i': [],
    'z': [],
    'y': [],
    'total': []
}

for target in targets:
    data_target_counts['target'].append(target)
    selection_target = (visits['target_name'] == target)
    data_target_counts['total'].append(np.sum(selection_target))
    for band in bands:
        selection_band = (visits['band'] == band)
        data_target_counts[band].append(np.sum(selection_band & selection_target))

df_target_counts = pd.DataFrame(data_target_counts)
df_target_counts

### Distribution of visits by target and band on most recent day

In [ ]:
most_recent_day_obs = np.unique(visits['day_obs'])[-1]
selection_day_obs = (visits['day_obs'] == most_recent_day_obs)
print(f'Observations on day_obs = {most_recent_day_obs}')

In [ ]:
data_target_counts = {
    'target': [],
    'u': [],
    'g': [],
    'r': [],
    'i': [],
    'z': [],
    'y': [],
    'total': []
}

for target in targets:
    data_target_counts['target'].append(target)
    selection_target = (visits['target_name'] == target)
    data_target_counts['total'].append(np.sum(selection_target & selection_day_obs))
    for band in bands:
        selection_band = (visits['band'] == band)
        data_target_counts[band].append(np.sum(selection_band & selection_target & selection_day_obs))

df_target_counts = pd.DataFrame(data_target_counts)
df_target_counts

### Distribution of per-ccd delivered image quality by target and band

In [ ]:
visits_query = f'''
    SELECT 
    ccdvisit1_quicklook.psf_sigma,
    visit1.visit_id,
    visit1.band,
    visit1.day_obs,
    visit1.target_name,
    visit1.exp_midpt_mjd
    FROM
    cdb_{instrument}.ccdvisit1_quicklook AS ccdvisit1_quicklook,
    cdb_{instrument}.ccdvisit1 AS ccdvisit1, 
    cdb_{instrument}.visit1 AS visit1 
    WHERE 
    ccdvisit1.ccdvisit_id = ccdvisit1_quicklook.ccdvisit_id
    AND ccdvisit1.visit_id = visit1.visit_id 
    AND visit1.science_program = 'BLOCK-365';
'''

ccdvisits = client.query(visits_query).to_pandas()

In [ ]:
ccdvisits["psf_fwhm"]=ccdvisits["psf_sigma"]*2.355*0.2

In [ ]:
targets_ccdvisits = np.unique(ccdvisits['target_name'])

for target in targets_ccdvisits:
    selection_target = (ccdvisits['target_name'] == target)

    plt.figure()
    for band in bands:
        selection_band = (ccdvisits['band'] == band)
        if np.any(selection_target & selection_band):    
            plt.hist(ccdvisits["psf_fwhm"][selection_target & selection_band], bins=np.linspace(0., 3., 41), color=bands[band], alpha=0.5, label=band)
            plt.hist(ccdvisits["psf_fwhm"][selection_target & selection_band], bins=np.linspace(0., 3., 41), color=bands[band], linewidth=2, histtype='step')
    plt.xlim(0.5, 2.5)
    plt.xlabel('PSF FWHM (arcsec)')
    plt.ylabel('Number of CCDs')
    plt.title(f'{target}')
    plt.legend(loc='upper right')
    plt.tight_layout()

### Distribution of per-ccd delivered image quality by target and band on most recent day

In [ ]:
targets_ccdvisits = np.unique(ccdvisits['target_name'])

selection_day_obs = (ccdvisits['day_obs'] == most_recent_day_obs)

for target in targets_ccdvisits:
    selection_target = (ccdvisits['target_name'] == target)

    if np.any(selection_target & selection_day_obs):
    
        plt.figure()
        for band in bands:
            selection_band = (ccdvisits['band'] == band)
            if np.any(selection_target & selection_band & selection_day_obs):    
                plt.hist(ccdvisits["psf_fwhm"][selection_target & selection_band & selection_day_obs], bins=np.linspace(0., 3., 41), color=bands[band], alpha=0.5, label=band)
                plt.hist(ccdvisits["psf_fwhm"][selection_target & selection_band & selection_day_obs], bins=np.linspace(0., 3., 41), color=bands[band], linewidth=2, histtype='step')
        plt.xlim(0.5, 2.5)
        plt.xlabel('PSF FWHM (arcsec)')
        plt.ylabel('Number of CCDs')
        plt.title(f'{target} (day_obs = {most_recent_day_obs})')
        plt.legend(loc='upper right')
        plt.tight_layout()

### Distribution of visit airmass

In [ ]:
plt.figure()
for target in targets:
    selection_target = (visits['target_name'] == target)
    # plt.hist(visits['airmass'][selection_target].values, bins=np.linspace(1., 2., 41), alpha=0.5)# label=target)
    plt.hist(visits['airmass'][selection_target].values, bins=np.linspace(1., 2., 41), linewidth=2, histtype='step', label=target)
plt.legend()
plt.xlabel('Airmass')
plt.ylabel('Counts')

### Distribution of visit airmass by target and time

In [ ]:
for target in targets:
    selection_target = (visits['target_name'] == target)
    
    plt.figure()
    for band in bands:
        selection_band = (visits['band'] == band) 
        if np.any(selection_target & selection_band):
            plt.scatter(dayObsToMJD(visits['day_obs'][selection_target & selection_band]), 
                        visits['airmass'][selection_target & selection_band], 
                        c=bands[band], marker='_', label=band)
    plt.legend(loc='lower left')
    plt.xlim(np.min(xticks) - 0.5, np.max(xticks) + 0.5)
    plt.xticks(xticks, xtick_labels, rotation=45.)
    plt.ylabel('Airmass')
    plt.ylim(1.5, 1.0)
    plt.grid()
    plt.title(target)
    plt.tight_layout()